In [54]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Dataset de Train

Se carga la data de entrenamiento para encontrar la similitud de productos del dataset de test

In [63]:
train_data = pd.read_csv("../dataset/items_titles.csv")

Se tiene que preprocesar la data para tener tener todos los titulos en minúsuculo y eliminar caracteres especiales que puedan alterar la similitud de una manera importante.

In [64]:
train_data['ITE_ITEM_TITLE'] = train_data['ITE_ITEM_TITLE'].str.lower()
train_data['ITE_ITEM_TITLE'] = train_data['ITE_ITEM_TITLE'].str.replace('[^\w\s]','')

/var/folders/76/xcgbvlyj3p173vhr81m7mn780000gn/T/ipykernel_2306/497468271.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['ITE_ITEM_TITLE'] = train_data['ITE_ITEM_TITLE'].str.replace('[^\w\s]','')


In [65]:
train_data.head(10)

,ITE_ITEM_TITLE
0,tênis ascension posh masculino preto e vermelho
1,tenis para caminhada super levinho spider corr...
2,tênis feminino le parc hocks blackice original...
3,tênis olympikus esportivo academia nova tendên...
4,inteligente led bicicleta tauda luz usb bicicl...
5,tênis casual masculino zarato 941 preto 632
6,tênis infantil ortopasso conforto jogging
7,tv samsung qled 8k q800t semi nova
8,tênis usthemp short temático maria viralata 2
9,sapatênis west coast urban couro masculino


Se vectoriza cada uno de los titulos de los items, para transformar estos titulos a vectores. Pra esto se usa TfidVectorizer de sklearn y se usa el método fit_transform para obtener esa matriz de que contiene todos los vectores de los items

In [66]:
vectorizer = TfidfVectorizer()
vectorized_train_data = vectorizer.fit_transform(train_data["ITE_ITEM_TITLE"])

Se usa cosine_similarity de sklearn para tener un valor de similaridad entre cada uno de los titulos de items en el dataset, es practicamente el producto punto entre cada uno de los titulos de items que estan representados en vectores.

In [67]:
cosine_similarities = cosine_similarity(vectorized_train_data)

In [68]:
results = []
for i, row in enumerate(train_data.itertuples()):
    title = row.ITE_ITEM_TITLE
    similarity_scores = cosine_similarities[i]
    most_similar_indices = similarity_scores.argsort()[-2:][::-1]
    most_similar_title = train_data.iloc[most_similar_indices[1]].ITE_ITEM_TITLE
    similarity = similarity_scores[most_similar_indices[1]]
    results.append({'Item': title, 'Most similar item': most_similar_title, 'Similarity score': similarity})
    
# Create a DataFrame from the list of results
items_similarity = pd.DataFrame(results)

In [61]:
items_similarity.head(5)

,Item,Most similar item,Similarity score
0,tênis ascension posh masculino preto e vermelho,tênis masculino ascension caminhada barato pre...,0.586383
1,tenis para caminhada super levinho spider corr...,tênis masculino caminhada levinho spider acade...,0.695795
2,tênis feminino le parc hocks blackice original...,tênis feminino hocks skatista le parc branco e...,0.718189
3,tênis olympikus esportivo academia nova tendên...,tênis olympikus esportivo academia nova tendên...,0.667771
4,inteligente led bicicleta tauda luz usb bicicl...,bicicleta,0.319583


## Dataset de Test

Para este dataset se realiza el mismo proceso, pero se utiliza el objeto de vectorizado con el cual se hizo el fit con los datos de entrenamiento y adicional se guardan los resultados en un archivo csv con cada par de items similares junto con su score.

In [69]:
# Load the test dataset
test_data = pd.read_csv("../dataset/items_titles_test.csv")

# Preprocess the test dataset
test_data['ITE_ITEM_TITLE'] = test_data['ITE_ITEM_TITLE'].str.lower()

# Vectorize the test dataset using the same vectorizer object
vectorized_test_data = vectorizer.transform(test_data['ITE_ITEM_TITLE'])

# Calculate cosine similarity between the test dataset and the training dataset
test_cosine_similarities = cosine_similarity(vectorized_test_data)

In [70]:
results = []
for i, row in enumerate(test_data.itertuples()):
    title = row.ITE_ITEM_TITLE
    similarity_scores = test_cosine_similarities[i]   
    most_similar_indices = similarity_scores.argsort()[-2:][::-1]
    most_similar_title = test_data.iloc[most_similar_indices[1]].ITE_ITEM_TITLE
    similarity = similarity_scores[most_similar_indices[1]]
    results.append({'Item': title, 'Most similar item': most_similar_title, 'Similarity score': similarity})
results_df = pd.DataFrame(results)

In [72]:
results_df.head(10)

,Item,Most similar item,Similarity score
0,tênis olympikus esporte valente - masculino kids,tênis olympikus masculino valente - kids infan...,0.836405
1,bicicleta barra forte samy c/ 6 marchas cubo c...,bicicleta leve dobra roda de rolamento fácil de,0.280689
2,tênis usthemp slip-on temático - labrador 2,tênis usthemp volare temático - labrador 2,0.711222
3,tênis casual feminino moleca tecido tie dye,tênis kolosh casual tie dye,0.663838
4,tênis star baby sapatinho conforto + brinde,tenis infantil meninas kidy conforto macio bab...,0.392061
5,tênis oakley frequency 3.0 preto/marrom,tênis oakley blocked preto,0.538233
6,tênis jogging feminino premium super lançament...,vizzano tênis casuai,0.426179
7,under armour hovr phantom 2 conexão bluetooth ...,tênis under armour hovr phantom 2 masculino,0.786370
8,tenis infantil feminino menina criança moça,tênis masculino infantil criança- 015998,0.713371
9,tênis labellamafia saturn 6 cores disponíveis,tênis unissex labellamafia saturn 2 vermelho,0.771643


se ordena por score de similitud para encontrar los pares de productos mas similares en el dataset de test

In [76]:
results_df = results_df.sort_values(by=["Similarity score"], ascending=False)
results_df.head(5)

,Item,Most similar item,Similarity score
9454,tênis activtta corrida masculino esportivo cat...,tênis activtta corrida masculino esportivo cat...,1.0
7627,tênis esportivo activtta masculino cataluna co...,tênis activtta corrida masculino esportivo cat...,1.0
138,tênis mizuno wave prophecy 6 - ref***000017,tênis mizuno wave prophecy 6 - ref***0000057,1.0
7948,sapatilha absolute mtb wild preto/vermelho,sapatilha absolute mtb wild preto/vermelho,1.0
7957,tênis lindo,lindo tênis,1.0


In [77]:
# Save the DataFrame to a CSV file
results_df.to_csv('items_similarity_test.csv', index=False)